In [5]:
# %pip install numpy
# %pip install mcts-simple

In [3]:
from our_packit import triangular_mode, hexagonal_mode

board = triangular_mode.game_logic.get_board(4)
triangular_mode.display.print_board(board) 

        /0\
      /0\0/0\
    /0\0/0\0/0\
  /0\0/0\0/0\0/0\


In [55]:
from typing import Any
from mcts_simple import Game
import numpy as np

from our_packit.triangular_mode.display import print_board
import our_packit.triangular_mode.game_logic as tgc 

class TriangularPackit(Game):
    def __init__(self, board_size):
        self.board = tgc.get_board(board_size)
        self.players = [0, 1]
        self.turn = 1
        self._update_actions()

    def _update_actions(self):
        self.possible_actions_list = tgc.get_possible_moves(self.board, self.turn)

    def render(self):
        print_board(self.board)

    def get_state(self):
        return np.copy(self.board)

    def number_of_players(self):
        return len(self.players)

    def current_player(self):
        return (self.turn + 1) % 2  

    def possible_actions(self):
        return list(range(len(self.possible_actions_list)))

    def take_action(self, action: int):
        self.board = tgc.place_polygon(self.board, self.possible_actions_list[action])
        self.turn += 1
        self._update_actions()

    def has_outcome(self):
        return len(self.possible_actions_list) == 0

    def winner(self):
        return [self.players[self.turn % 2]]
    


In [56]:
from mcts_simple import UCT, MCTS

game = TriangularPackit(4)

tree = UCT(game, allow_transpositions=False, training=True)
tree.self_play(iterations=1000)
tree.save("triangularPackit402.mcts")

Training: 100%|██████████| 1000/1000 [01:23<00:00, 12.01it/s]


In [66]:
import random
from copy import deepcopy
from tqdm import tqdm

wins = {
    0: 0,
    1: 0
}
game = TriangularPackit(4)
tree = MCTS(game, allow_transpositions = False, training = False)
tree.load("triangularPackit402.mcts")
num_games = 1000
for i in tqdm(range(num_games)):
    human_player = i % 2
    game = TriangularPackit(4)
    # tree = MCTS(game, allow_transpositions = False, training = False)
    # tree.load("triangularPackit401.mcts")
    node = tree.root
    while not game.has_outcome():
        # game.render()
        actions = game.possible_actions()
        if game.current_player() == human_player:
            # print("Possible actions:")
            # for i, a in zip(actions, game.possible_actions_list):
            #     print(f'{i}: {a}')
            # action = int(input("> "))
            action = random.choice(actions)
            assert action in actions
            if node is not None:
                node = node.children[action] if action in node.children else None
        else:
            if node is not None and len(node.children) > 0:
                action = node.choose_best_action(tree.training)
                node = node.children[action]
            else:
                action = random.choice(actions)
                node = None
        game.take_action(action)
    # game.render()
    if 1-human_player in game.winner():
        wins[1-human_player] += 1
    
print(f'zaczynając: {wins[0] / (num_games/2)}, nie zaczynając: {wins[1] / (num_games/2)}')

100%|██████████| 1000/1000 [00:03<00:00, 275.27it/s]

zaczynając: 0.226, nie zaczynając: 0.978


In [ ]:
import random
from copy import deepcopy
from tqdm import tqdm

wins = {
    0: 0,
    1: 0
}
game = TriangularPackit(4)
tree = MCTS(game, allow_transpositions = False, training = False)
tree.load("triangularPackit401.mcts")
tree2 = MCTS(game, allow_transpositions = False, training = False)
tree2.load("triangularPackit402.mcts")
num_games = 1000
for i in tqdm(range(num_games)):
    human_player = i % 2
    game = TriangularPackit()
    # tree = MCTS(game, allow_transpositions = False, training = False)
    # tree.load("triangularPackit401.mcts")
    game = TriangularPackit()
    node = tree.root
    while not game.has_outcome():
        # game.render()
        actions = game.possible_actions()
        if game.current_player() == human_player:
            # print("Possible actions:")
            # for i, a in zip(actions, game.possible_actions_list):
            #     print(f'{i}: {a}')
            # action = int(input("> "))
            action = random.choice(actions)
            assert action in actions
            if node is not None:
                node = node.children[action] if action in node.children else None
        else:
            if node is not None and len(node.children) > 0:
                action = node.choose_best_action(tree.training)
                node = node.children[action]
            else:
                action = random.choice(actions)
                node = None
        game.take_action(action)
    # game.render()
    if 1-human_player in game.winner():
        wins[1-human_player] += 1
    
print(f'zaczynając: {wins[0] / (num_games/2)}, nie zaczynając: {wins[1] / (num_games/2)}')

In [31]:
import random
from copy import deepcopy

human_player = 0 # X

game = TriangularPackit()
tree = MCTS(game, allow_transpositions = False, training = False)
tree.load("triangularPackit.mcts")
game = TriangularPackit()
node = tree.root
while not game.has_outcome():
    game.render()
    actions = game.possible_actions()
    if game.current_player() == human_player:
        print("Possible actions:")
        for i, a in zip(actions, game.possible_actions_list):
            print(f'{i}: {a}')
        action = int(input("> "))
        assert action in actions
        if node is not None:
            node = node.children[action] if action in node.children else None
    else:
        if node is not None and len(node.children) > 0:
            action = node.choose_best_action(tree.training)
            node = node.children[action]
        else:
            action = random.choice(actions)
            node = None
    game.take_action(action)
game.render()

          /0\
        /0\0/0\
      /0\0/0\0/0\
    /0\0/0\0/0\0/0\
  /0\0/0\0/0\0/0\0/0\
Possible actions:
0: [[1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
1: [[0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
2: [[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
3: [[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
4: [[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]]
5: [[0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
6: [[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
7: [[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
8: [[0 0 0 0 0 0 0 0